In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-06 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# SWNU Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swnu_hidden_dim_sizes_and_sig_depths": swnu_hidden_dim_sizes_and_sig_depths,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "BiLSTM": True,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## umap

In [12]:
(
    swnu_network_umap_kfold_5,
    best_swnu_network_umap_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_5_kfold_best_model.csv


In [13]:
swnu_network_umap_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3571270/1104105075.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.634610   
                                                       0.0005         0.626948   
                                                       0.0010         0.623372   
                                          0.2          0.0001         0.637676   
                                                       0.0005         0.643167   
                                                       0.0010         0.621456   
                           (512, 512)     0.1          0.0001         0.637612   
                                                       0.0005         0.624202   
                                                       0.0010         0.614751   
                                          0.2          0.0001         0.638953   
                                                       0.0005         0.628991   
                                                       0.0010         0.621137   
           (12,)           (256, 256)     0.1          0.0001         0.631226   
                                                       0.0005         0.629885   
                                                       0.0010         0.620498   
                                          0.2          0.0001         0.626820   
                                                       0.0005         0.635760   
                                                       0.0010         0.617561   
                           (512, 512)     0.1          0.0001         0.640358   
                                                       0.0005         0.621264   
                                                       0.0010         0.614432   
                                          0.2          0.0001         0.632439   
                                                       0.0005         0.627650   
                                                       0.0010         0.622733   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.509872   
                                                       0.0005         0.510112   
                                                       0.0010         0.502148   
                                          0.2          0.0001         0.516230   
                                                       0.0005         0.513259   
                                                       0.0010         0.504421   
                           (512, 512)     0.1          0.0001         0.513750   
                                                       0.0005         0.511786   
                                                       0.0010         0.493971   
                                          0.2          0.0001         0.506532   
                                                       0.0005         0.514297   
                                                       0.0010         0.495324   
           (12,)           (256, 256)     0.1          0.0001         0.498417   
                                                       0.0005         0.500838   
                                                       0.0010         0.496384   
                                          0.2          0.0001         0.495513   
                                                       0.0005         0.512391   
                                                       0.0010         0.501798   
                           (512, 512)     0.1          0.0001         0.509148   
                                                       0.0005         0.496871   
                                                       0.0010         0.502381   
                                          0.2      

In [14]:
swnu_network_umap_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3571270/1104105075.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.634610   
                                                       0.0005         0.626948   
                                                       0.0010         0.623372   
                                          0.2          0.0001         0.637676   
                                                       0.0005         0.643167   
                                                       0.0010         0.621456   
                           (512, 512)     0.1          0.0001         0.637612   
                                                       0.0005         0.624202   
                                                       0.0010         0.614751   
                                          0.2          0.0001         0.638953   
                                                       0.0005         0.628991   
                                                       0.0010         0.621137   
           (12,)           (256, 256)     0.1          0.0001         0.631226   
                                                       0.0005         0.629885   
                                                       0.0010         0.620498   
                                          0.2          0.0001         0.626820   
                                                       0.0005         0.635760   
                                                       0.0010         0.617561   
                           (512, 512)     0.1          0.0001         0.640358   
                                                       0.0005         0.621264   
                                                       0.0010         0.614432   
                                          0.2          0.0001         0.632439   
                                                       0.0005         0.627650   
                                                       0.0010         0.622733   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.509872   
                                                       0.0005         0.510112   
                                                       0.0010         0.502148   
                                          0.2          0.0001         0.516230   
                                                       0.0005         0.513259   
                                                       0.0010         0.504421   
                           (512, 512)     0.1          0.0001         0.513750   
                                                       0.0005         0.511786   
                                                       0.0010         0.493971   
                                          0.2          0.0001         0.506532   
                                                       0.0005         0.514297   
                                                       0.0010         0.495324   
           (12,)           (256, 256)     0.1          0.0001         0.498417   
                                                       0.0005         0.500838   
                                                       0.0010         0.496384   
                                          0.2          0.0001         0.495513   
                                                       0.0005         0.512391   
                                                       0.0010         0.501798   
                           (512, 512)     0.1          0.0001         0.509148   
                                                       0.0005         0.496871   
                                                       0.0010         0.502381   
                                          0.2      

In [15]:
best_swnu_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.617816,0.501858,"[0.7411076091850519, 0.43332029722330856, 0.33...",0.494582,"[0.7703588143525741, 0.41189591078066917, 0.30...",0.512788,"[0.7139965297860035, 0.4570957095709571, 0.367...",None,0.662492,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.619157,0.512129,"[0.7428307123034228, 0.4562071591774562, 0.337...",0.502371,"[0.7955746367239102, 0.42362093352192365, 0.28...",0.532714,"[0.6966454598033546, 0.49422442244224424, 0.40...",None,0.648683,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.635632,0.521369,"[0.7528735632183908, 0.4764772299585999, 0.334...",0.514280,"[0.7891566265060241, 0.4380622837370242, 0.315...",0.532807,"[0.7197802197802198, 0.5222772277227723, 0.356...",None,0.658478,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


in path (with speaker): 0.5143628505176907

in path (no speaker): 0.522911065406166

in input (no speaker): 0.5125295634373418

both (no speaker): 0.5151677550731781

(w/ time_encoding) in path (with speaker): 0.5186878521492474

(w/ time_encoding) in path (no speaker): 0.5191477428145815

(w/ time_encoding) in input (no speaker): 0.5126337587709865

(w/ time_encoding) both (no speaker):  0.5197361095991764

In [16]:
best_swnu_network_umap_kfold_5["f1"].mean()

0.511785569768353

In [17]:
best_swnu_network_umap_kfold_5["precision"].mean()

0.503744383168738

In [18]:
best_swnu_network_umap_kfold_5["recall"].mean()

0.5261031844460713

In [19]:
np.stack(best_swnu_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.74560396, 0.4553349 , 0.33441785])

In [20]:
np.stack(best_swnu_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.78503003, 0.42452638, 0.30167675])

In [21]:
np.stack(best_swnu_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.71014074, 0.49119912, 0.3769697 ])

# w=11

In [22]:
size = 11

## umap

In [23]:
(
    swnu_network_umap_kfold_11,
    best_swnu_network_umap_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_11_kfold_best_model.csv


In [24]:
swnu_network_umap_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3571270/2463364505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.630460   
                                                       0.0005         0.619285   
                                                       0.0010         0.628352   
                                          0.2          0.0001         0.635185   
                                                       0.0005         0.631481   
                                                       0.0010         0.624777   
                           (512, 512)     0.1          0.0001         0.638250   
                                                       0.0005         0.623563   
                                                       0.0010         0.627075   
                                          0.2          0.0001         0.633525   
                                                       0.0005         0.603065   
                                                       0.0010         0.627650   
           (12,)           (256, 256)     0.1          0.0001         0.637165   
                                                       0.0005         0.615900   
                                                       0.0010         0.610664   
                                          0.2          0.0001         0.625160   
                                                       0.0005         0.624649   
                                                       0.0010         0.622989   
                           (512, 512)     0.1          0.0001         0.629438   
                                                       0.0005         0.627778   
                                                       0.0010         0.608685   
                                          0.2          0.0001         0.635121   
                                                       0.0005         0.627714   
                                                       0.0010         0.629630   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.490055   
                                                       0.0005         0.498026   
                                                       0.0010         0.512144   
                                          0.2          0.0001         0.492091   
                                                       0.0005         0.504601   
                                                       0.0010         0.505631   
                           (512, 512)     0.1          0.0001         0.497894   
                                                       0.0005         0.505139   
                                                       0.0010         0.504702   
                                          0.2          0.0001         0.493888   
                                                       0.0005         0.495845   
                                                       0.0010         0.503630   
           (12,)           (256, 256)     0.1          0.0001         0.503340   
                                                       0.0005         0.503144   
                                                       0.0010         0.509005   
                                          0.2          0.0001         0.491853   
                                                       0.0005         0.502696   
                                                       0.0010         0.498190   
                           (512, 512)     0.1          0.0001         0.498998   
                                                       0.0005         0.493905   
                                                       0.0010         0.485609   
                                          0.2      

In [25]:
swnu_network_umap_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3571270/2463364505.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.630460   
                                                       0.0005         0.619285   
                                                       0.0010         0.628352   
                                          0.2          0.0001         0.635185   
                                                       0.0005         0.631481   
                                                       0.0010         0.624777   
                           (512, 512)     0.1          0.0001         0.638250   
                                                       0.0005         0.623563   
                                                       0.0010         0.627075   
                                          0.2          0.0001         0.633525   
                                                       0.0005         0.603065   
                                                       0.0010         0.627650   
           (12,)           (256, 256)     0.1          0.0001         0.637165   
                                                       0.0005         0.615900   
                                                       0.0010         0.610664   
                                          0.2          0.0001         0.625160   
                                                       0.0005         0.624649   
                                                       0.0010         0.622989   
                           (512, 512)     0.1          0.0001         0.629438   
                                                       0.0005         0.627778   
                                                       0.0010         0.608685   
                                          0.2          0.0001         0.635121   
                                                       0.0005         0.627714   
                                                       0.0010         0.629630   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.490055   
                                                       0.0005         0.498026   
                                                       0.0010         0.512144   
                                          0.2          0.0001         0.492091   
                                                       0.0005         0.504601   
                                                       0.0010         0.505631   
                           (512, 512)     0.1          0.0001         0.497894   
                                                       0.0005         0.505139   
                                                       0.0010         0.504702   
                                          0.2          0.0001         0.493888   
                                                       0.0005         0.495845   
                                                       0.0010         0.503630   
           (12,)           (256, 256)     0.1          0.0001         0.503340   
                                                       0.0005         0.503144   
                                                       0.0010         0.509005   
                                          0.2          0.0001         0.491853   
                                                       0.0005         0.502696   
                                                       0.0010         0.498190   
                           (512, 512)     0.1          0.0001         0.498998   
                                                       0.0005         0.493905   
                                                       0.0010         0.485609   
                                          0.2      

In [26]:
best_swnu_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.635441,0.508926,"[0.760666277030976, 0.4267676767676768, 0.3393...",0.504425,"[0.7687536916715889, 0.43556701030927836, 0.30...",0.515809,"[0.7527472527472527, 0.4183168316831683, 0.376...",None,0.666827,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.615900,0.510634,"[0.7397386625672561, 0.44427244582043346, 0.34...",0.500460,"[0.7896291434197571, 0.41836734693877553, 0.29...",0.532216,"[0.6957779063042221, 0.4735973597359736, 0.427...",None,0.649968,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.633716,0.516873,"[0.7521574564723695, 0.4691358024691358, 0.329...",0.514492,"[0.7893231649189705, 0.41893644617380027, 0.33...",0.524991,"[0.7183342972816658, 0.533003300330033, 0.3236...",None,0.657033,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [27]:
best_swnu_network_umap_kfold_11["f1"].mean()

0.5121441926991973

In [28]:
best_swnu_network_umap_kfold_11["precision"].mean()

0.5064590967476486

In [29]:
best_swnu_network_umap_kfold_11["recall"].mean()

0.5243388528172269

In [30]:
np.stack(best_swnu_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75085413, 0.44672531, 0.33885314])

In [31]:
np.stack(best_swnu_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78256867, 0.42429027, 0.31251836])

In [32]:
np.stack(best_swnu_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.72228649, 0.4749725 , 0.37575758])

# w=20

In [33]:
size = 20

## umap

In [34]:
(
    swnu_network_umap_kfold_20,
    best_swnu_network_umap_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swnu_network_umap_focal_2_20_kfold_best_model.csv


In [35]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3571270/29893126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.616156   
                                                       0.0005         0.621520   
                                                       0.0010         0.630460   
                                          0.2          0.0001         0.620179   
                                                       0.0005         0.604534   
                                                       0.0010         0.610345   
                           (512, 512)     0.1          0.0001         0.613729   
                                                       0.0005         0.625862   
                                                       0.0010         0.633780   
                                          0.2          0.0001         0.622861   
                                                       0.0005         0.620115   
                                                       0.0010         0.612452   
           (12,)           (256, 256)     0.1          0.0001         0.621839   
                                                       0.0005         0.623180   
                                                       0.0010         0.619732   
                                          0.2          0.0001         0.617050   
                                                       0.0005         0.611750   
                                                       0.0010         0.616603   
                           (512, 512)     0.1          0.0001         0.618710   
                                                       0.0005         0.618646   
                                                       0.0010         0.620370   
                                          0.2          0.0001         0.621456   
                                                       0.0005         0.610792   
                                                       0.0010         0.616858   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.466171   
                                                       0.0005         0.496234   
                                                       0.0010         0.518782   
                                          0.2          0.0001         0.470320   
                                                       0.0005         0.486019   
                                                       0.0010         0.496068   
                           (512, 512)     0.1          0.0001         0.471706   
                                                       0.0005         0.496320   
                                                       0.0010         0.505073   
                                          0.2          0.0001         0.472559   
                                                       0.0005         0.496543   
                                                       0.0010         0.497554   
           (12,)           (256, 256)     0.1          0.0001         0.479423   
                                                       0.0005         0.490058   
                                                       0.0010         0.505524   
                                          0.2          0.0001         0.477271   
                                                       0.0005         0.477674   
                                                       0.0010         0.501749   
                           (512, 512)     0.1          0.0001         0.480210   
                                                       0.0005         0.485089   
                                                       0.0010         0.500886   
                                          0.2      

In [36]:
swnu_network_umap_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3571270/29893126.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_umap_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.616156   
                                                       0.0005         0.621520   
                                                       0.0010         0.630460   
                                          0.2          0.0001         0.620179   
                                                       0.0005         0.604534   
                                                       0.0010         0.610345   
                           (512, 512)     0.1          0.0001         0.613729   
                                                       0.0005         0.625862   
                                                       0.0010         0.633780   
                                          0.2          0.0001         0.622861   
                                                       0.0005         0.620115   
                                                       0.0010         0.612452   
           (12,)           (256, 256)     0.1          0.0001         0.621839   
                                                       0.0005         0.623180   
                                                       0.0010         0.619732   
                                          0.2          0.0001         0.617050   
                                                       0.0005         0.611750   
                                                       0.0010         0.616603   
                           (512, 512)     0.1          0.0001         0.618710   
                                                       0.0005         0.618646   
                                                       0.0010         0.620370   
                                          0.2          0.0001         0.621456   
                                                       0.0005         0.610792   
                                                       0.0010         0.616858   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.466171   
                                                       0.0005         0.496234   
                                                       0.0010         0.518782   
                                          0.2          0.0001         0.470320   
                                                       0.0005         0.486019   
                                                       0.0010         0.496068   
                           (512, 512)     0.1          0.0001         0.471706   
                                                       0.0005         0.496320   
                                                       0.0010         0.505073   
                                          0.2          0.0001         0.472559   
                                                       0.0005         0.496543   
                                                       0.0010         0.497554   
           (12,)           (256, 256)     0.1          0.0001         0.479423   
                                                       0.0005         0.490058   
                                                       0.0010         0.505524   
                                          0.2          0.0001         0.477271   
                                                       0.0005         0.477674   
                                                       0.0010         0.501749   
                           (512, 512)     0.1          0.0001         0.480210   
                                                       0.0005         0.485089   
                                                       0.0010         0.500886   
                                          0.2      

In [37]:
best_swnu_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.615326,0.496320,"[0.7425995492111195, 0.42693974005513985, 0.31...",0.489091,"[0.7729121050985299, 0.4084400904295403, 0.285...",0.507863,"[0.7145748987854251, 0.4471947194719472, 0.361...",None,0.664258,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.600192,0.486747,"[0.7259946156147173, 0.39333862014274384, 0.34...",0.479468,"[0.7518587360594795, 0.3786259541984733, 0.307...",0.497637,"[0.7018507807981492, 0.40924092409240925, 0.38...",None,0.656390,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.615517,0.505137,"[0.7321622882014612, 0.4748700173310226, 0.308...",0.501163,"[0.7915966386554621, 0.40944411237298267, 0.30...",0.520252,"[0.6810294968189705, 0.5651815181518152, 0.314...",None,0.654464,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [38]:
best_swnu_network_umap_kfold_20["f1"].mean()

0.496068218773479

In [39]:
best_swnu_network_umap_kfold_20["precision"].mean()

0.4899069575616617

In [40]:
best_swnu_network_umap_kfold_20["recall"].mean()

0.5085837951445038

In [41]:
np.stack(best_swnu_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.73358548, 0.43171613, 0.32290305])

In [42]:
np.stack(best_swnu_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.77212249, 0.39883672, 0.29876166])

In [43]:
np.stack(best_swnu_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.69915173, 0.47387239, 0.35272727])

# w=35

In [44]:
size = 35

## umap

In [ ]:
(
    swnu_network_umap_kfold_35,
    best_swnu_network_umap_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_umap_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_umap_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_35

In [ ]:
best_swnu_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_35["precision_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## umap

In [ ]:
(
    swnu_network_umap_kfold_80,
    best_swnu_network_umap_kfold_80,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_umap_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_umap_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_80

In [ ]:
best_swnu_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_80["recall_scores"]).mean(axis=0)

# w=110

In [ ]:
size = 110

## umap

In [ ]:
(
    swnu_network_umap_kfold_110,
    best_swnu_network_umap_kfold_110,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swnu_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swnu_network_umap_kfold_110.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
swnu_network_umap_kfold_110.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_umap_kfold_110

In [ ]:
best_swnu_network_umap_kfold_110["f1"].mean()

In [ ]:
best_swnu_network_umap_kfold_110["precision"].mean()

In [ ]:
best_swnu_network_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swnu_network_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_umap_kfold_110["recall_scores"]).mean(axis=0)